In [1]:
import requests, boto3, pandas as pd, os, sys, pprint, logging
from pathlib import Path
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

# Initialize log config.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


# Making the current directory in which this file is in discoverable to python.
# Commenting it here because it will not work in jupyter notebook. It will work in lambda though.
#sys.path.append(os.path.join(os.path.dirname(__file__)))

# Below should be used only in jupyter notebook
sys.path.append('./')


REDDIT_AUTH_URL = 'https://www.reddit.com/api/v1/access_token'
REDDIT_ACCOUNTS_TABLE_NAME = 'RedditAccountsTable-dev'
DAILY_UPLOADS_TABLE_NAME = "DailyUploadsTable-dev"
REDDIT_API_URL_TOP = "https://oauth.reddit.com/r/placeholder_value/top"
REDDIT_API_URL_SORT = "https://oauth.reddit.com/r/placeholder_value/sort"

ddb = boto3.client("dynamodb", region_name="ap-south-1")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
from entities.GatherUrls import GatherUrls
from entities.RedditAccount import RedditAccount
subreddit='funny'

In [3]:
reddit_account = RedditAccount(subreddit=subreddit, ddb=ddb, logger=logger)
gather_urls = GatherUrls(subreddit=subreddit, logger=logger)
REDDIT_API_URL_TOP = REDDIT_API_URL_TOP.replace('placeholder_value','funny')

In [4]:
reddit_account.fetch_and_update_account_details(REDDIT_ACCOUNTS_TABLE_NAME)
reddit_account.authenticate_with_api()

INFO:root:Fetched and updated the following account details:

INFO:root:{ 'PK': 'funny',
  'email_address': 'mugblsxlqpoqbclaqi@bptfp.net',
  'password': 'Abcd@12349',
  'personal_use_script': 'epsNZpypuQ1IngadWDnlGg',
  'secret_key': 'yG4Ej57nkBMIdFzpWkzhNXChnsiluw',
  'username': 'mugblsxlqpoqbclaqi'}


In [5]:
reddit_account.fetch_and_update_access_token(REDDIT_AUTH_URL)

In [6]:
posts =  reddit_account.fetch_posts_as_json(REDDIT_API_URL_TOP)

In [7]:
gather_urls.parse_posts(posts)

INFO:root:For funny on date: 2021-08-14
INFO:root:('Post with title: I prototype stupid ideas and today I made a zip off bathing '
 'suit with towel pant legs. with duration: 38s is added to eligible posts\n')
INFO:root:('Post with title: Kid runs on football pitch. Mom takes him down and carries '
 'him back. with duration: 25s is added to eligible posts\n')
INFO:root:("Post with title: Dude; Where's my bike? with duration: 23s is added to "
 'eligible posts\n')
INFO:root:('Post with title: Yo! Hold the door! with duration: 12s is added to eligible '
 'posts\n')
INFO:root:("Post with title: Garbage service isn't as safe as you think with duration: "
 '28s is added to eligible posts\n')
INFO:root:('Post with title: and the Oscar goes to.... with duration: 16s is added to '
 'eligible posts\n')
INFO:root:('Post with title: I re-edited the Matrix throat chop video to be something '
 'other than potato quality. Enjoy! with duration: 35s is added to eligible '
 'posts\n')
INFO:root:('Post 

In [8]:
DAILY_UPLOADS_TABLE_NAME = "DailyUploadsTable-dev"
TransactItems=[
                {
                    "Put": {
                        "TableName": DAILY_UPLOADS_TABLE_NAME,
                        "Item": gather_urls.serialize_to_item(),
                    }
                },
                {
                    "Update": {
                        "TableName": DAILY_UPLOADS_TABLE_NAME,
                        "Key": {
                            "PK": {"S": '2013-12-12'},
                            "SK": {"S": "todays_subreddits_count"},
                        },
                        "ConditionExpression": "attribute_exists(PK) and attribute_exists(SK)",
                        "UpdateExpression": "SET #count = #count + :inc",
                        "ExpressionAttributeNames": {"#count": "count"},
                        "ExpressionAttributeValues": {":inc": {"N": "1"}},
                    }
                },
            ]

INFO:root:('Serialized item is:\n'
 "{'PK': {'S': '2021-08-14'}, 'SK': {'S': 'funny'}, 'posts': {'L': [{'M': "
 "{'title': {'S': 'I prototype stupid ideas and today I made a zip off bathing "
 "suit with towel pant legs.'}, 'url': {'S': "
 "'https://v.redd.it/4nex55lbp6h71'}, 'upvote_ratio': {'N': '0.81'}, 'ups': "
 "{'N': '61550'}, 'author': {'S': 'rightcoastguy'}, 'name': {'S': "
 "'t3_p3tc8p'}, 'total_awards_received': {'N': '88'}}}, {'M': {'title': {'S': "
 "'Kid runs on football pitch. Mom takes him down and carries him back.'}, "
 "'url': {'S': 'https://v.redd.it/dyb2i0y666h71'}, 'upvote_ratio': {'N': "
 "'0.98'}, 'ups': {'N': '5844'}, 'author': {'S': 'justgossiping'}, 'name': "
 "{'S': 't3_p3r9r9'}, 'total_awards_received': {'N': '6'}}}, {'M': {'title': "
 '{\'S\': "Dude; Where\'s my bike?"}, \'url\': {\'S\': '
 "'https://v.redd.it/fevwhi5nu7h71'}, 'upvote_ratio': {'N': '0.95'}, 'ups': "
 "{'N': '2212'}, 'author': {'S': 'WrickyB'}, 'name': {'S': 't3_p3xkk1'}, "
 "'total_awards_r

In [9]:
import ddb_helpers

resp = ddb_helpers.transact_write_items(ddb=ddb,TransactItems=TransactItems,logger=logger)

--- Logging error ---
Traceback (most recent call last):
  File "/home/sourabh/work/personal/whodoesntlovereddit/images/ddb_helpers/__init__.py", line 32, in transact_write_items
    resp = ddb.transact_write_items(TransactItems=TransactItems)
  File "/home/sourabh/work/personal/whodoesntlovereddit/.venv/lib/python3.8/site-packages/botocore/client.py", line 386, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/sourabh/work/personal/whodoesntlovereddit/.venv/lib/python3.8/site-packages/botocore/client.py", line 705, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.TransactionCanceledException: An error occurred (TransactionCanceledException) when calling the TransactWriteItems operation: Transaction cancelled, please refer cancellation reasons for specific reasons [None, ConditionalCheckFailed]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/us

TransactionCanceledException: An error occurred (TransactionCanceledException) when calling the TransactWriteItems operation: Transaction cancelled, please refer cancellation reasons for specific reasons [None, ConditionalCheckFailed]